<a href="https://colab.research.google.com/github/BME6938-SPR2023/final_project/blob/main/PelvicSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install dependencies

In [1]:
!pip install -q transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 15.5 MB/s eta 0:00:00


## import libraries

In [2]:
#NEED to delect unused libraries
import os as os
from glob import glob
import shutil
from tqdm import tqdm
import numpy as np
import nibabel as nib
import tensorflow as tf
import matplotlib.pyplot as plt
from torch import nn
import evaluate
import multiprocessing
import warnings
warnings.filterwarnings('ignore')

os.environ['TORCH_USE_CUDA_DSA'] = '1'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import torch
import cv2
import PIL
from torch.utils.data import Dataset, DataLoader
from skimage.transform import resize, rotate
# import SimpleITK as sitk
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
import torchvision.transforms as transforms
from PIL import Image
from transformers import SegformerFeatureExtractor,SegformerImageProcessor, AutoImageProcessor, TrainingArguments, Trainer

## HF Login

In [17]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [18]:
hf_username = "ashiyakatuka11"

## Data preparation: 
#### convert nii to png and remove slices without matching labels
- Resized and rotated images 
- Reduced the dataset by only including images that match with segmentation masks with 1 or both labels
- There is an option for converting from .nii to .png

In [ ]:
#data cleaning 1: use stacked data/ labels to get label list
def clean_data(input_folder, output_folder, image_processor): 
  #first remove matched images without labels

  img_folder_path = '/stacked_sliced/train' #switch "train" to "test"
  lbl_folder_path = '/prostate_femur_sliced/train'

  img_dir = input_folder + img_folder_path
  label_dir = input_folder + lbl_folder_path

  labels_paths = []

  img_paths = sorted([os.path.join( img_dir, x) for x in os.listdir( img_dir)])
  labels_paths = sorted([os.path.join(label_dir, x) for x in os.listdir(label_dir)])

  for i_img,i_mask in zip(img_paths,labels_paths):
    img_rename = os.path.splitext(os.path.basename(i_img))[0]
    label_rename = os.path.splitext(os.path.basename(i_mask))[0]

    Reading_Img_NII = nib.load(i_img)
    Reading_Label_NII = nib.load(i_mask)

    img_arr = Reading_Img_NII.get_fdata()
    mask_arr = Reading_Label_NII.get_fdata()
    
     # Define the new size for the resized image
    new_size = (512, 512)
    
    # Resize the image using the resize() function from skimage
    img_resized = resize(img_arr, new_size, anti_aliasing=True)
    mask_resized = resize(mask_arr, new_size, anti_aliasing=True)

    # Rotate the image using the rotate() function from skimage
    angle = 90
    img_resized = rotate(img_resized , angle)
    mask_resized = rotate(mask_resized, angle)

    # Normalized 
    img_arr_norm = (img_resized- img_resized.min()) / (img_resized.max() - img_resized.min())
    pil_image = Image.fromarray((img_arr_norm * 255).astype(np.uint8))

    label_arr_norm = (mask_resized - mask_resized.min()) / (mask_resized .max() - mask_resized.min())
    pil_label = Image.fromarray((label_arr_norm* 255).astype(np.uint8)) 

    pil_image_T = np.stack([pil_image ] * 3, axis=-1).transpose((2,1,0))
    encoded_inputs = image_processor(pil_image_T, pil_label, return_tensors="pt")

    labels = encoded_inputs["labels"].squeeze().unique()
    if len(labels) > 2:
      labels_paths.append(i_mask )
      print(i_img)
      #save stack slices with labels
      pil_image.save(f'/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack2{img_folder_path}/{img_rename}.png')
      pil_label.save(f'/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack2{lbl_folder_path}/{label_rename}.png')
  return(labels_paths)

In [ ]:
def clean_data2(input_folder,output_folder):
  img_folder_path = '/T2_sliced/test' #switch "train" to "test"
  lbl_folder_path = '/prostate_femur_sliced/test'
  newlbl_folder_path = '/prostate_femur_sliced/test'

  img_dir = input_folder + img_folder_path
  label_dir = input_folder + lbl_folder_path
  newlabel_dir = output_folder + lbl_folder_path

  img_paths = sorted([os.path.join(img_dir, x) for x in os.listdir(img_dir)])
  labels_paths = sorted([os.path.join(label_dir, x) for x in os.listdir(label_dir)])
  newlabels_paths = sorted([os.path.join(newlabel_dir, x) for x in os.listdir(newlabel_dir)])

  for i_img,i_mask in zip(img_paths,labels_paths):
    img_rename = os.path.splitext(os.path.basename(i_img))[0]
    label_rename = os.path.splitext(os.path.basename(i_mask))[0]
    for new_mask in newlabels_paths:
      newlabel_rename = os.path.splitext(os.path.basename(new_mask))[0]
      if (label_rename == newlabel_rename):
        Reading_Img_NII = nib.load(i_img)
        # Reading_Label_NII = nib.load(i_mask)

        img_arr = Reading_Img_NII.get_fdata()
        # mask_arr = Reading_Label_NII.get_fdata()
        
        # Define the new size for the resized image
        new_size = (512, 512)
        
        # Resize the image using the resize() function from skimage
        img_resized = resize(img_arr, new_size, anti_aliasing=True)
        # mask_resized = resize(mask_arr, new_size, anti_aliasing=True)

        # Rotate the image using the rotate() function from skimage
        angle = 90
        img_resized = rotate(img_resized , angle)
        # mask_resized = rotate(mask_resized, angle)

        img_arr_norm = (img_resized- img_resized.min()) / (img_resized.max() - img_resized.min())
        pil_image = Image.fromarray((img_arr_norm * 255).astype(np.uint8))

        # label_arr_norm = (mask_resized - mask_resized.min()) / (mask_resized .max() - mask_resized.min())
        # pil_label = Image.fromarray((label_arr_norm* 255).astype(np.uint8))

        print(i_img)
        #save stack slices with labels
        pil_image.save(f'/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack2{img_folder_path}/{img_rename}.png')
        # pil_image.save(f'/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack2{lbl_folder_path}/{label_rename}.png')
  print("Done")


In [ ]:
folder_path = "/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack" #folder with original images
output_path = "/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack2" #folder where new images will be saved

In [ ]:
image_processor = SegformerImageProcessor(use_python_backend=False)


In [ ]:
clean_data(folder_path,output_path, image_processor) #Run ONLY TWICE for stacked/label train and stacked/label test


In [ ]:
clean_data2(folder_path,output_path) #Run 4 times for T2 and dCT2  train and T2 and dCT2l test

### .nii reduced

In [ ]:
#data cleaning 1: use stacked data/ labels to get label list
import shutil
def reduced_data(input_folder, output_path, image_processor): 
  #first remove matched images without labels
 
  img_folder_path = '/stacked_sliced/train' #switch "train" to "test"
  lbl_folder_path = '/prostate_femur_sliced/train'

  # new_folder_path = newfolder/img_folder_path
  # !mkdir /content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack_reduced/stacked_sliced/test
  # !mkdir /content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack_reduced/stacked_sliced/test

  img_dir = input_folder + img_folder_path
  label_dir = input_folder + lbl_folder_path

  out_img_dir = output_path + img_folder_path
  out_lbl_dir = output_path + lbl_folder_path

  labels_paths = []

  img_paths = sorted([os.path.join( img_dir, x) for x in os.listdir( img_dir)])
  labels_paths = sorted([os.path.join(label_dir, x) for x in os.listdir(label_dir)])

  for i_img,i_mask in zip(img_paths,labels_paths):
    img_rename = os.path.splitext(os.path.basename(i_img))
    label_rename = os.path.splitext(os.path.basename(i_mask))

    Reading_Img_NII = nib.load(i_img)
    Reading_Label_NII = nib.load(i_mask)

    img_arr = Reading_Img_NII.get_fdata()
    mask_arr = Reading_Label_NII.get_fdata()
    
     # Define the new size for the resized image
    new_size = (512, 512)
    
    # Resize the image using the resize() function from skimage
    img_resized = resize(img_arr, new_size, anti_aliasing=True)
    mask_resized = resize(mask_arr, new_size, anti_aliasing=True)

    # Rotate the image using the rotate() function from skimage
    angle = 90
    img_resized = rotate(img_resized , angle)
    mask_resized = rotate(mask_resized, angle)

    img_arr_norm = (img_resized- img_resized.min()) / (img_resized.max() - img_resized.min())
    pil_image = Image.fromarray((img_arr_norm * 255).astype(np.uint8))

    label_arr_norm = (mask_resized - mask_resized.min()) / (mask_resized .max() - mask_resized.min())
    pil_label = Image.fromarray((label_arr_norm* 255).astype(np.uint8)) 

    pil_image_T = np.stack([pil_image ] * 3, axis=-1).transpose((2,1,0))
    encoded_inputs = image_processor(pil_image_T, pil_label, return_tensors="pt")

    labels = encoded_inputs["labels"].squeeze().unique()
    if len(labels) > 2:
      labels_paths.append(i_mask )
      print(i_img)
      #save stack slices with labels
      shutil.copy(i_img, out_img_dir )
      # shutil.copy(i_mask,out_lbl_dir )
  return(labels_paths)

In [ ]:
def reduced_data2(input_folder,output_path):
  img_folder_path = '/stacked_sliced/train' #switch "train" to "test"
  lbl_folder_path = '/prostate_femur_sliced/train'
  newlbl_folder_path = '/prostate_femur_sliced/test'

  img_dir = input_folder + img_folder_path
  label_dir = input_folder + lbl_folder_path
  newlabel_dir = output_path + lbl_folder_path

  out_img_dir = output_path + img_folder_path
  # out_lbl_dir = output_path + lbl_folder_path

  img_paths = sorted([os.path.join(img_dir, x) for x in os.listdir(img_dir)])
  labels_paths = sorted([os.path.join(label_dir, x) for x in os.listdir(label_dir)])
  newlabels_paths = sorted([os.path.join(newlabel_dir, x) for x in os.listdir(newlabel_dir)])

  for i_img,i_mask in zip(img_paths,labels_paths):
    img_rename = os.path.splitext(os.path.basename(i_img))
    label_rename = os.path.splitext(os.path.basename(i_mask))
    for new_mask in newlabels_paths:
      newlabel_rename = os.path.splitext(os.path.basename(new_mask))
      if (label_rename == newlabel_rename):
        Reading_Img_NII = nib.load(i_img)
        # Reading_Label_NII = nib.load(i_mask)

        img_arr = Reading_Img_NII.get_fdata()
        # mask_arr = Reading_Label_NII.get_fdata()
        
        # Define the new size for the resized image
        new_size = (512, 512)
        
        # Resize the image using the resize() function from skimage
        img_resized = resize(img_arr, new_size, anti_aliasing=True)
        # mask_resized = resize(mask_arr, new_size, anti_aliasing=True)

        # Rotate the image using the rotate() function from skimage
        angle = 90
        img_resized = rotate(img_resized , angle)
        # mask_resized = rotate(mask_resized, angle)

        img_arr_norm = (img_resized- img_resized.min()) / (img_resized.max() - img_resized.min())
        pil_image = Image.fromarray((img_arr_norm * 255).astype(np.uint8))

        # label_arr_norm = (mask_resized - mask_resized.min()) / (mask_resized .max() - mask_resized.min())
        # pil_label = Image.fromarray((label_arr_norm* 255).astype(np.uint8))

        print(i_img)
        #save stack slices with labels
        shutil.copy(i_img,out_img_dir )
        # pil_image.save(f'/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack2{lbl_folder_path}/{label_rename}.png')
  print("Done")

In [ ]:
folder_path = "/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack"
output_path = "/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack_reduced"

In [ ]:
image_processor = SegformerImageProcessor(use_python_backend=False)

In [ ]:
reduced_data(folder_path,output_path, image_processor) #Run ONLY TWICE for stacked/label train and stacked/label test

In [ ]:
reduced_data2(folder_path,output_path) #Run 4 times for T2 and dCT2  train and T2 and dCT2l test

## Load the data 

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# folder_path = "/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack2" #new directory with new .png images 

In [4]:

folder_path = "/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack_reduced"  #new directory with new reduced .nii images 

### check labels 

In [ ]:
#sample image/label
img_path = "/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack2/stacked_sliced/train/P3_04_stack_Z48.png"
lbl_path = "/content/drive/MyDrive/Colab_Notebooks/datasets/GoldA_ZStack2/prostate_femur_sliced/train/3_04_prostate_femur_Z48.png"
img = Image.open(img_path)
label= Image.open(lbl_path)
label_arr = np.array(label)
print(np.unique(label_arr))

#visualize image/mask

# figure,axis = plt.subplots(1,2,figsize=(10,10))
# # Plot_Color_Op = axis[0].imshow(img)
# axis[0].imshow(img)
# axis[0].set_xlabel(img.size)
# axis[0].set_ylabel(img.size)
# axis[0].set_title("IMAGE")

# axis[1].imshow(label)
# axis[1].set_xlabel(label.size)
# axis[1].set_ylabel(label.size)
# axis[1].set_title("MASK")

[  0 254 255]


In [ ]:
#sample image/label
img_path = folder_path + "/stacked_sliced/train/P1_02_stack_Z48.nii"
lbl_path = folder_path + "/prostate_femur_sliced/train/3_04_prostate_femur_Z48.nii"


Reading_Img_NII = nib.load(img_path)
Reading_Label_NII = nib.load(lbl_path)

img_arr = Reading_Img_NII.get_fdata()
mask_arr = Reading_Label_NII.get_fdata()
print(np.unique(mask_arr))

#visualize image/mask

# # Plot_Color_Op = axis[0].imshow(img)
# axis[0].imshow(img)
# axis[0].set_xlabel(img.size)
# axis[0].set_ylabel(img.size)
# axis[0].set_title("IMAGE")

# axis[1].imshow(label)
# axis[1].set_xlabel(label.size)
# axis[1].set_ylabel(label.size)
# axis[1].set_title("MASK")

[0. 2.]


### Get the unique labels

In [ ]:
#data cleaning 1: use stacked data/ labels to get label list
def unique_data(input_folder): 

  lbl_folder_path = '/prostate_femur_sliced/test'
  label_dir = input_folder + lbl_folder_path

  labels_paths = []
  labels_paths = sorted([os.path.join(label_dir, x) for x in os.listdir(label_dir)])

  for i_mask in labels_paths:
    Reading_Label_NII = nib.load(i_mask)
    mask_arr = Reading_Label_NII.get_fdata()
    
    print(np.unique(mask_arr))
  return("done")

In [ ]:
unique_data(folder_path)

[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1.]
[0. 1.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.]
[0. 2.]
[0. 2.]
[0. 2.]
[0. 2.]
[0. 2.]
[0. 2.]


'done'

### load image/label directories

In [5]:
## directory paths for all datasets

##train
tr_stk_img_dir=folder_path + '/stacked_sliced/train'
tr_t2_img_dir=folder_path + '/T2_sliced/train'
tr_dct_img_dir=folder_path + '/dCT2_sliced/train'
tr_labels_dir=folder_path +'/prostate_femur_sliced/train'
# tr_newlabels_dir=folder_path +'/prostate/train'

##test
ts_stk_img_dir=folder_path + '/stacked_sliced/test'
ts_t2_img_dir=folder_path + '/T2_sliced/test'
ts_dct_img_dir=folder_path + '/dCT2_sliced/test'
ts_labels_dir=folder_path +'/prostate_femur_sliced/test'

In [6]:
tr_stk_img_paths = sorted([os.path.join(tr_stk_img_dir, x) for x in os.listdir(tr_stk_img_dir)])
tr_t2_img_paths = sorted([os.path.join(tr_t2_img_dir, x) for x in os.listdir(tr_t2_img_dir)])
tr_dct_img_paths = sorted([os.path.join(tr_dct_img_dir, x) for x in os.listdir(tr_dct_img_dir)])
tr_labels_paths = sorted([os.path.join(tr_labels_dir, x) for x in os.listdir(tr_labels_dir)])
# tr_newlabels_paths = sorted([os.path.join(tr_newlabels_dir, x) for x in os.listdir(tr_newlabels_dir)])

ts_stk_img_paths = sorted([os.path.join(ts_stk_img_dir, x) for x in os.listdir(ts_stk_img_dir)])
ts_t2_img_paths = sorted([os.path.join(ts_t2_img_dir, x) for x in os.listdir(ts_t2_img_dir)])
ts_dct_img_paths = sorted([os.path.join(ts_dct_img_dir, x) for x in os.listdir(ts_dct_img_dir)])
ts_labels_paths = sorted([os.path.join(ts_labels_dir, x) for x in os.listdir(ts_labels_dir)])

In [7]:
#check to make sure all images match with labels
print(len(tr_stk_img_paths))
print(len(tr_t2_img_paths))
print(len(tr_dct_img_paths))
print(len(tr_labels_paths))
print("***")
print(len(ts_stk_img_paths))
print(len(ts_t2_img_paths))
print(len(ts_dct_img_paths))
print(len(ts_labels_paths))

299
299
299
299
***
51
51
51
51


## Data Preprocessing
To make them compatible with the SegFormer model from Hugging Face Transformers, we:

- Normalize the images with the mean and standard deviation used during pre-training SegFormer.
- Transpose the images such that they are in "channels_first" format. 

### pelvic segmentation class

In [8]:
### .nii
class NiftiPelvicSegmentationDataset(torch.utils.data.Dataset):
    """Image (pelvic) segmentation dataset."""

    def __init__(self, img_paths, mask_paths, train=True):
        """
        Args:

        """
        self.img_paths = img_paths
        self.mask_paths = mask_paths
        # self.transform = transform
        self.train = train

        assert len(self.img_paths) == len(self.mask_paths), "check that number of images equals number of segmentation masks "

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx): 

        #Load the Nifti image      
        img = nib.load(self.img_paths[idx])
        mask = nib.load(self.mask_paths[idx])

        # Convert the Nifti image to a NumPy array
        img_arr = img.get_fdata()
        mask_arr = mask.get_fdata()

        # Define the new size for the resized image
        if (img_arr.shape != (512, 512)):
          new_size = (512, 512)
          # Resize the image using the resize() function from skimage
          img_arr= resize(img_arr, new_size, anti_aliasing=True)
          mask_arr = resize(mask_arr, new_size, anti_aliasing=True)

        #Normalize the image
        img_arr = (img_arr - img_arr.min()) / (img_arr.max() - img_arr.min())
        img_arr = np.stack([img_arr ] * 3, axis=-1).transpose((2,1,0))

        # if self.transform is not None:
        #     img_arr = self.transform(img_arr)
        #     mask_arr = self.transform(mask_arr)

        # Convert to PyTorch tensors
        img_tensor = torch.from_numpy(img_arr).unsqueeze(0) # Add channel dimension
        mask_tensor = torch.from_numpy(mask_arr).unsqueeze(0) # Add channel dimension

        # pad both image and segmentation map to same size
        inputs = {"pixel_values": img_tensor.squeeze_().float(), "labels": mask_tensor.squeeze_().long()}

        return inputs

In [9]:
stk_train_dataset = NiftiPelvicSegmentationDataset(img_paths=tr_stk_img_paths, mask_paths=tr_labels_paths)
stk_valid_dataset = NiftiPelvicSegmentationDataset(img_paths=ts_stk_img_paths, mask_paths=ts_labels_paths, train=False)

In [10]:
t2_train_dataset = NiftiPelvicSegmentationDataset(img_paths=tr_t2_img_paths, mask_paths=tr_labels_paths)
t2_valid_dataset = NiftiPelvicSegmentationDataset(img_paths=ts_t2_img_paths, mask_paths=ts_labels_paths, train=False)

In [11]:
dct_train_dataset = NiftiPelvicSegmentationDataset(img_paths=tr_dct_img_paths, mask_paths=tr_labels_paths)
dct_valid_dataset = NiftiPelvicSegmentationDataset(img_paths=ts_dct_img_paths, mask_paths=ts_labels_paths, train=False)

### check data 

In [ ]:
print("# of training data:", len(train_dataset))
print("# of test data:", len(valid_dataset))

# of training data: 299
# of test data: 51


In [ ]:
encoded_inputs = train_dataset[0]
print(encoded_inputs["pixel_values"].shape)
print(encoded_inputs["labels"].shape)
print(encoded_inputs["labels"].squeeze().unique())

torch.Size([3, 512, 512])
torch.Size([512, 512])
tensor([0, 1])


### Get the labels 

In [ ]:
# unique_labels = set()
unique_pixels_list = []
#train data
for i in range(len(train_dataset)):
  unique_pixels = train_dataset[i]["labels"].squeeze().unique()
  if str(unique_pixels) not in unique_pixels_list:
    unique_pixels_list.append(str(unique_pixels))

#test data
for i in range(len(valid_dataset)):
  unique_pixels = valid_dataset[i]["labels"].squeeze().unique()
  if str(unique_pixels) not in unique_pixels_list:
    unique_pixels_list.append(str(unique_pixels))

print(len(unique_pixels_list))


3


In [ ]:
batch["labels"].shape

## Finetuning the model

In [12]:
#set labels
import json
from huggingface_hub import hf_hub_download

# hf_dataset_identifier = "Cats-Can/image_data"

# filename = "id2label.json"
# id2label = json.load(open(hf_hub_download(repo_id=hf_dataset_identifier, filename=filename, repo_type="dataset"), "r"))
id2label = {0: 'Background',  1: 'Prostate', 2: 'Femur'} #uncomment to use if having problems with HF
id2label = {int(k): v for k, v in id2label.items()}
label2id = {v: k for k, v in id2label.items()}

num_labels = len(id2label)
print("Id2label:", id2label)
print(num_labels)

Id2label: {0: 'Background', 1: 'Prostate', 2: 'Femur'}
3


In [13]:
# load model
from transformers import SegformerForSemanticSegmentation

In [14]:
# define model
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/mit-b0",
                                                         num_labels=3, 
                                                         id2label=id2label, 
                                                         label2id=label2id,
)

Some weights of the model checkpoint at nvidia/mit-b0 were not used when initializing SegformerForSemanticSegmentation: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.linear_fuse.weight', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.batch_norm.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.batch_nor

In [15]:
from datasets import load_metric

metric = load_metric("mean_iou")
# metric2 = load_metric("precision")
# metric3 = load_metric("recall")
# metric4 = load_metric("f1")
# metric4 = load_metric("erntkn/dice_coefficient")

In [16]:
from datasets import list_metrics
# list_metrics()

In [ ]:
from datasets import Metric

In [19]:
from transformers import TrainingArguments

epochs = 300
lr = 0.00006
batch_size = 4

hub_model_id = "segformer-b0-finetuned-pelvic-segmentation-stacked"

training_args = TrainingArguments(
    "segformer-b0-finetuned-pelvic-outputs",
    learning_rate=lr,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=40,
    eval_steps=40,
    logging_steps=1,
    eval_accumulation_steps=5,
    load_best_model_at_end=True,
    # push_to_hub=True,
    # hub_model_id=hub_model_id,
    # hub_strategy="end",
)

In [20]:
from sklearn.metrics import precision_score,recall_score,f1_score

def compute_metrics(eval_pred):
  with torch.no_grad():
    logits, labels = eval_pred
    logits_tensor = torch.from_numpy(logits)

    # scale the logits to the size of the label
    logits_tensor = nn.functional.interpolate(
        logits_tensor,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
    ).argmax(dim=1)

    pred_labels = logits_tensor.detach().cpu().numpy()
    metrics = metric._compute(
            predictions=pred_labels,
            references=labels,
            num_labels=len(id2label),
            ignore_index=0,

        )
  
    # metrics2 = metric2._compute(
    #         predictions=pred_labels,
    #         references=labels,
    #         # num_labels=len(id2label),
    #         # ignore_index=0,
    #     )
    

    # add per category metrics as individual key-value pairs
    per_category_accuracy = metrics.pop("per_category_accuracy").tolist()  
    per_category_iou = metrics.pop("per_category_iou").tolist()


    metrics.update({f"accuracy_{id2label[i]}": v for i, v in enumerate(per_category_accuracy)})
    metrics.update({f"iou_{id2label[i]}": v for i, v in enumerate(per_category_iou)})

    # metrics["precision"] = 
    # print("Precision:", metrics2["precision"])
    # print("Recall:", metrics["recall"])
    # print("Precision:", metrics["precision"])
    # print("F1:", metrics["f1"])
    # print("DICE:", metrics["erntkn/dice_coefficient"])

    return metrics

In [21]:
#stacked 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=stk_train_dataset,
    eval_dataset=stk_valid_dataset,
    compute_metrics=compute_metrics,
)

In [22]:
trainer.train()

Step,Training Loss,Validation Loss,Mean Iou,Mean Accuracy,Overall Accuracy,Accuracy Background,Accuracy Prostate,Accuracy Femur,Iou Background,Iou Prostate,Iou Femur
40,0.658500,0.537546,0.019103,0.028889,0.032679,nan,0.005499,0.052279,0.000000,0.005497,0.051811
80,0.418700,0.407625,0.103886,0.155829,0.181082,nan,0.000000,0.311657,0.000000,0.000000,0.311657
120,0.281400,0.262497,0.081512,0.122269,0.142083,nan,0.000000,0.244537,0.000000,0.000000,0.244537
160,0.177000,0.179142,0.048474,0.072712,0.084495,nan,0.000000,0.145423,0.000000,0.000000,0.145423
200,0.119800,0.112971,0.008562,0.012842,0.014924,nan,0.000000,0.025685,0.000000,0.000000,0.025685
240,0.097400,0.086545,0.014916,0.022393,0.026023,nan,0.000000,0.044787,0.000000,0.000000,0.044749
280,0.086300,0.071643,0.014947,0.022421,0.026054,nan,0.000000,0.044842,0.000000,0.000000,0.044842
320,0.057500,0.058970,0.061616,0.092684,0.107705,nan,0.000000,0.185369,0.000000,0.000000,0.184847
360,0.052700,0.058911,0.022602,0.033937,0.037357,nan,0.012835,0.055039,0.000000,0.012835,0.054972
400,0.043500,0.056033,0.023334,0.035055,0.039635,nan,0.006790,0.063319,0.000000,0.006790,0.063213


TrainOutput(global_step=22500, training_loss=0.007567016396160599, metrics={'train_runtime': 6511.6171, 'train_samples_per_second': 13.775, 'train_steps_per_second': 3.455, 'total_flos': 1.572366387511296e+18, 'train_loss': 0.007567016396160599, 'epoch': 300.0})

In [23]:
from transformers import TrainingArguments

epochs = 200
lr = 0.00006
batch_size = 4

hub_model_id = "segformer-b0-finetuned-pelvic-segmentation-t2"

training_args = TrainingArguments(
    "segformer-b0-finetuned-pelvic-outputs",
    learning_rate=lr,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=40,
    eval_steps=40,
    logging_steps=1,
    eval_accumulation_steps=5,
    load_best_model_at_end=True,
    # push_to_hub=True,
    # hub_model_id=hub_model_id,
    # hub_strategy="end",
)

In [24]:
#T2
trainer2 = Trainer(
    model=model,
    args=training_args,
    train_dataset=t2_train_dataset,
    eval_dataset=t2_valid_dataset,
    compute_metrics=compute_metrics,
)

In [25]:
trainer2.train()

Step,Training Loss,Validation Loss,Mean Iou,Mean Accuracy,Overall Accuracy,Accuracy Background,Accuracy Prostate,Accuracy Femur,Iou Background,Iou Prostate,Iou Femur
40,0.013100,0.050097,0.115834,0.173882,0.165669,nan,0.224565,0.123200,0.000000,0.224565,0.122938
80,0.005300,0.050457,0.073448,0.110306,0.111320,nan,0.104050,0.116562,0.000000,0.104050,0.116296
120,0.014700,0.051033,0.106566,0.160155,0.163664,nan,0.138500,0.181809,0.000000,0.138500,0.181198
160,0.012300,0.048116,0.158064,0.237518,0.233936,nan,0.259623,0.215413,0.000000,0.259623,0.214569
200,0.007400,0.050582,0.161551,0.242803,0.233586,nan,0.299678,0.185928,0.000000,0.299678,0.184975
240,0.005900,0.053874,0.106872,0.160614,0.158191,nan,0.175563,0.145664,0.000000,0.175563,0.145053
280,0.012500,0.055370,0.108700,0.163170,0.154309,nan,0.217851,0.108490,0.000000,0.217851,0.108250
320,0.008800,0.051084,0.142794,0.214608,0.212992,nan,0.224580,0.204635,0.000000,0.224580,0.203803
360,0.007000,0.060330,0.094995,0.142631,0.138730,nan,0.166707,0.118556,0.000000,0.166707,0.118278
400,0.008800,0.055950,0.135730,0.203957,0.201893,nan,0.216697,0.191218,0.000000,0.216697,0.190494


TrainOutput(global_step=15000, training_loss=0.0026494285892520565, metrics={'train_runtime': 4270.6633, 'train_samples_per_second': 14.003, 'train_steps_per_second': 3.512, 'total_flos': 1.048244258340864e+18, 'train_loss': 0.0026494285892520565, 'epoch': 200.0})

In [ ]:
#dcT
# trainer3 = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=dct_train_dataset,
#     eval_dataset=dct_valid_dataset,
#     compute_metrics=compute_metrics,
# )

In [ ]:
# trainer3.train()

## Evaluation

## Get model saved in HF

In [ ]:
chkpt = "ashiyakatuka11/segformer-b0-finetuned-pelvic-segmentation"

### Evaluate stacked model

In [ ]:
trainer.evaluate(dct_valid_dataset)

In [ ]:
trainer.evaluate(t2_valid_dataset)

{'eval_loss': 0.36775073409080505,
 'eval_mean_iou': 0.0,
 'eval_mean_accuracy': 0.0,
 'eval_overall_accuracy': 0.0,
 'eval_accuracy_Background': nan,
 'eval_accuracy_Prostate': 0.0,
 'eval_accuracy_Femur': 0.0,
 'eval_iou_Background': 0.0,
 'eval_iou_Prostate': 0.0,
 'eval_iou_Femur': 0.0,
 'eval_runtime': 27.482,
 'eval_samples_per_second': 1.856,
 'eval_steps_per_second': 0.473,
 'epoch': 1.0}

### Evalutate T2 model

In [ ]:
trainer2.evaluate(dct_valid_dataset)

{'eval_loss': 0.38671594858169556,
 'eval_mean_iou': 0.0,
 'eval_mean_accuracy': 0.0,
 'eval_overall_accuracy': 0.0,
 'eval_accuracy_Background': nan,
 'eval_accuracy_Prostate': 0.0,
 'eval_accuracy_Femur': 0.0,
 'eval_iou_Background': 0.0,
 'eval_iou_Prostate': 0.0,
 'eval_iou_Femur': 0.0,
 'eval_runtime': 25.8839,
 'eval_samples_per_second': 1.97,
 'eval_steps_per_second': 0.502,
 'epoch': 1.0}

In [ ]:
trainer2.evaluate(stk_valid_dataset)

{'eval_loss': 0.39052000641822815,
 'eval_mean_iou': 0.0,
 'eval_mean_accuracy': 0.0,
 'eval_overall_accuracy': 0.0,
 'eval_accuracy_Background': nan,
 'eval_accuracy_Prostate': 0.0,
 'eval_accuracy_Femur': 0.0,
 'eval_iou_Background': 0.0,
 'eval_iou_Prostate': 0.0,
 'eval_iou_Femur': 0.0,
 'eval_runtime': 27.0455,
 'eval_samples_per_second': 1.886,
 'eval_steps_per_second': 0.481,
 'epoch': 1.0}

### Evalutate dCT model

In [ ]:
trainer3.evaluate(stk_valid_dataset)

In [ ]:
trainer3.evaluate(t2_valid_dataset)

## Visualize results (TBD)

In [ ]:

Predict_Mask = prediction_image

figure,axis = plt.subplots(1,3,figsize=(10,10))
Plot_Color_Op = axis[0].imshow(img)
axis[0].set_xlabel(img.shape)
axis[0].set_ylabel(img.size)
axis[0].set_title("IMAGE")

axis[1].imshow(label)
axis[1].set_xlabel(label.shape)
axis[1].set_ylabel(label.size)
axis[1].set_title("MASK")
axis[2].imshow(Predict_Mask)
axis[2].set_xlabel(Predict_Mask.shape)
axis[2].set_ylabel(Predict_Mask.size)
axis[2].set_title("PREDICTION")

In [ ]:
# from datasets import load_dataset

# dataset = load_dataset("Cats-Can/pelvic_images")

In [ ]:
!pip install -q pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 20.4 MB/s eta 0:00:00


In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger

In [ ]:
class SegformerFinetuner(pl.LightningModule):
    
    def __init__(self, id2label, train_dataloader=None,  test_dataloader=None, metrics_interval=100):
        super(SegformerFinetuner, self).__init__()
        self.id2label = id2label
        self.metrics_interval = metrics_interval
        self.train_dl = train_dataloader
        self.test_dl = test_dataloader
        
        self.num_classes = len(id2label.keys())
        self.label2id = {v:k for k,v in self.id2label.items()}
        
        self.model = SegformerForSemanticSegmentation.from_pretrained(
            "nvidia/segformer-b0-finetuned-ade-512-512", 
            return_dict=False, 
            num_labels=self.num_classes,
            id2label=self.id2label,
            label2id=self.label2id,
            ignore_mismatched_sizes=True,
        )
        
        self.train_mean_iou = load_metric("mean_iou")
        self.test_mean_iou = load_metric("mean_iou")
        
    def forward(self, images, masks):
        outputs = self.model(pixel_values=images, labels=masks)
        return(outputs)
    
    def training_step(self, batch, batch_nb):
        
        images, masks = batch['pixel_values'], batch['labels']
        
        outputs = self(images, masks)
        
        loss, logits = outputs[0], outputs[1]
        
        upsampled_logits = nn.functional.interpolate(
            logits, 
            size=masks.shape[-2:], 
            mode="bilinear", 
            align_corners=False
        )

        predicted = upsampled_logits.argmax(dim=1)

        self.train_mean_iou.add_batch(
            predictions=predicted.detach().cpu().numpy(), 
            references=masks.detach().cpu().numpy()
        )
        if batch_nb % self.metrics_interval == 0:

            metrics = self.train_mean_iou.compute(
                num_labels=self.num_classes, 
                ignore_index=255, 
                reduce_labels=False,
            )
            
            metrics = {'loss': loss, "mean_iou": metrics["mean_iou"], "mean_accuracy": metrics["mean_accuracy"]}
            
            for k,v in metrics.items():
                self.log(k,v)
            
            return(metrics)
        else:
            return({'loss': loss})
      

    
    def test_step(self, batch, batch_nb):
        
        images, masks = batch['pixel_values'], batch['labels']
        
        outputs = self(images, masks)
        
        loss, logits = outputs[0], outputs[1]
        
        upsampled_logits = nn.functional.interpolate(
            logits, 
            size=masks.shape[-2:], 
            mode="bilinear", 
            align_corners=False
        )
        
        predicted = upsampled_logits.argmax(dim=1)
        
        self.test_mean_iou.add_batch(
            predictions=predicted.detach().cpu().numpy(), 
            references=masks.detach().cpu().numpy()
        )
            
        return({'test_loss': loss})
    
    def test_epoch_end(self, outputs):
        metrics = self.test_mean_iou.compute(
              num_labels=self.num_classes, 
              ignore_index=255, 
              reduce_labels=False,
          )
       
        avg_test_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        test_mean_iou = metrics["mean_iou"]
        test_mean_accuracy = metrics["mean_accuracy"]

        metrics = {"test_loss": avg_test_loss, "test_mean_iou":test_mean_iou, "test_mean_accuracy":test_mean_accuracy}
        
        for k,v in metrics.items():
            self.log(k,v)
        
        return metrics
    
    def configure_optimizers(self):
        return torch.optim.Adam([p for p in self.parameters() if p.requires_grad], lr=2e-05, eps=1e-08)
    
    def train_dataloader(self):
        return self.train_dl

    
    def test_dataloader(self):
        return self.test_dl

In [ ]:
batch_size = 4
num_workers = 2
train_dataloader = DataLoader(stk_train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_dataloader = DataLoader(stk_valid_dataset, batch_size=batch_size, num_workers=num_workers)

segformer_finetuner = SegformerFinetuner(
    id2label, 
    train_dataloader=train_dataloader, 
    test_dataloader=test_dataloader, 
    metrics_interval=10,
)

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([3, 256, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
early_stop_callback = EarlyStopping(
    monitor="val_loss", 
    min_delta=0.00, 
    patience=10, 
    verbose=False, 
    mode="min",
)

checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor="val_loss")

trainer = pl.Trainer(
    # gpus=1, 
    callbacks=[early_stop_callback, checkpoint_callback],
    max_epochs=1,
    val_check_interval=len(train_dataloader),
)
trainer.fit(segformer_finetuner)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                             | Params
-----------------------------------------------------------
0 | model | SegformerForSemanticSegmentation | 3.7 M 
-----------------------------------------------------------
3.7 M     Trainable params
0         Non-trainable params
3.7 M     Total params
14.860    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
trainer
